# Ensure Tables and Parameters are Set Up

## Load Bronze Data

In [0]:
-- LOAD BRONZE (limit to rolling N days, default 3)
DROP VIEW IF EXISTS lxc_status_bronze_view;

CREATE TEMPORARY VIEW lxc_status_bronze_view AS
SELECT
  -- Extract node name from meta.host URL
  regexp_extract(meta.host, 'https://([^.]+)', 1) as node_name,
  payload.data.vmid as vmid,
  payload.data.name as name,
  payload.data.status as status,
  payload.data.cpu as cpu,
  payload.data.cpus as cpus,
  payload.data.mem as mem,
  payload.data.maxmem as maxmem,
  payload.data.swap as swap,
  payload.data.maxswap as maxswap,
  payload.data.disk as disk,
  payload.data.maxdisk as maxdisk,
  payload.data.diskread as diskread,
  payload.data.diskwrite as diskwrite,
  payload.data.netin as netin,
  payload.data.netout as netout,
  payload.data.uptime as uptime,
  payload.data.tags as tags,
  CAST(meta.collected_at_utc AS TIMESTAMP) as ingestion_timestamp,
  dt as partition_date
FROM
  read_files(
    'abfss://homelab-telemetry@sa01homelabdata.dfs.core.windows.net/bronze/proxmox/lxc_status/*/*.jsonl',
    format => 'json'
  )
WHERE
  dt >= date_sub(current_date(), 3);

In [0]:
-- Verify how many rows were loaded into the bronze temp table
SELECT
  COUNT(*) as total_rows,
  COUNT(DISTINCT vmid) as distinct_containers,
  COUNT(DISTINCT node_name) as distinct_nodes,
  MIN(partition_date) as earliest_date,
  MAX(partition_date) as latest_date
FROM
  lxc_status_bronze_view;

## Write into Silver

In [0]:
-- MERGE BRONZE INTO SILVER
-- Dedup where vmid, node & ingestion_timestamp are matched
MERGE INTO
  homelab.silver.proxmox_lxc_status AS target
USING
  lxc_status_bronze_view AS source
ON
  target.node_name = source.node_name
  AND target.vmid = source.vmid
  AND target.ingestion_timestamp = source.ingestion_timestamp
WHEN NOT MATCHED THEN INSERT *;

## Write Gold

In [0]:
-- Add data to GOLD (processing ALL historical data)
-- Aggregates to datapoint by hour to get min/max/avg mem, cpu, disk, network traffic, and status by hour 
MERGE INTO homelab.gold.fact_proxmox_lxc_metrics_hourly AS target
USING (
  WITH hourly_aggregates AS (
    SELECT
      dn.node_key,
      ddh.date_hour_key,
      CAST(s.vmid AS STRING) as vmid,
      MAX(s.name) as name,
      MAX(s.partition_date) as report_date,
      MAX(ddh.report_hour) as report_hour,
      ROUND(AVG(s.cpu), 4) as avg_cpu_pct,
      ROUND(MAX(s.cpu), 4) as peak_cpu_pct,
      ROUND(MIN(s.cpu), 4) as min_cpu_pct,
      ROUND(AVG(CASE WHEN s.maxmem > 0 THEN s.mem / s.maxmem ELSE 0 END) * 100, 2) as avg_mem_pct,
      ROUND(MAX(CASE WHEN s.maxmem > 0 THEN s.mem / s.maxmem ELSE 0 END) * 100, 2) as peak_mem_pct,
      ROUND(MIN(CASE WHEN s.maxmem > 0 THEN s.mem / s.maxmem ELSE 0 END) * 100, 2) as min_mem_pct,
      ROUND(AVG(CASE WHEN s.maxdisk > 0 THEN s.disk / s.maxdisk ELSE 0 END) * 100, 2) as avg_disk_pct,
      ROUND(MAX(CASE WHEN s.maxdisk > 0 THEN s.disk / s.maxdisk ELSE 0 END) * 100, 2) as peak_disk_pct,
      ROUND(MIN(CASE WHEN s.maxdisk > 0 THEN s.disk / s.maxdisk ELSE 0 END) * 100, 2) as min_disk_pct,
      MAX(s.netin) as total_netin_bytes,
      MAX(s.netout) as total_netout_bytes,
      CASE WHEN COUNT_IF(s.status != 'running') > 0 THEN true ELSE false END as was_not_running,
      MAX(s.uptime) as uptime_seconds
    FROM
      homelab.silver.proxmox_lxc_status s
    INNER JOIN homelab.gold.dim_node dn ON s.node_name = dn.node_name
    INNER JOIN homelab.gold.dim_date_hour ddh ON s.partition_date = ddh.report_date AND hour(s.ingestion_timestamp) = ddh.report_hour
    GROUP BY
      dn.node_key,
      s.vmid,
      ddh.date_hour_key
  ),
  lagged_metrics AS (
    SELECT 
      *,
      -- Calculate the previous hour's totals once here
      LAG(total_netin_bytes) OVER (PARTITION BY node_key, vmid ORDER BY report_date, report_hour) as prev_netin_bytes,
      LAG(total_netout_bytes) OVER (PARTITION BY node_key, vmid ORDER BY report_date, report_hour) as prev_netout_bytes
    FROM hourly_aggregates
  )
  SELECT
    node_key,
    date_hour_key,
    vmid,
    name,
    report_date,
    report_hour,
    avg_cpu_pct,
    peak_cpu_pct,
    min_cpu_pct,
    avg_mem_pct,
    peak_mem_pct,
    min_mem_pct,
    avg_disk_pct,
    peak_disk_pct,
    min_disk_pct,
    total_netin_bytes,
    total_netout_bytes,
    
    -- Much cleaner logic for network calculations
    CASE 
      WHEN prev_netin_bytes IS NULL OR total_netin_bytes < prev_netin_bytes THEN NULL
      ELSE ROUND((total_netin_bytes - prev_netin_bytes) / 1000000.0, 2)
    END as netin_mb_per_hour,
    
    CASE 
      WHEN prev_netout_bytes IS NULL OR total_netout_bytes < prev_netout_bytes THEN NULL
      ELSE ROUND((total_netout_bytes - prev_netout_bytes) / 1000000.0, 2)
    END as netout_mb_per_hour,
    
    was_not_running,
    uptime_seconds
  FROM lagged_metrics
) AS source
ON target.node_key = source.node_key 
   AND target.vmid = source.vmid 
   AND target.date_hour_key = source.date_hour_key
WHEN NOT MATCHED THEN INSERT *;

In [0]:
select * from homelab.gold.fact_proxmox_lxc_metrics_hourly where node_key = 3 and name = 'homelab-collector' order by report_date, report_hour